<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto tem o intuito de estudar Modelos de Inferência Causal. </i>       
</p>  

# <font color='orange' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid orange;'>

In [ ]:
## Bibliotecas De Manipulação de Dados e Visualização
import pandas as pd 
import geopandas as gpd
import builtins as builtins
import matplotlib.pyplot as plt
import seaborn as sns 
import itertools
from IPython.display import display, Image

## Bibliotecas de Modelagem Matemática e Estatística
import numpy as np
import dowhy
from dowhy import CausalModel
from econml.dml import LinearDML
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.tsa import stattools
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import bernoulli, binom, poisson, geom, norm, chi2, f, chi2_contingency, normaltest, ttest_ind, ttest_rel, wilcoxon, mannwhitneyu, kruskal
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px 
import plotly.graph_objects as go
import plotnine as p


# Bibliotecas de Manipulação de Tempo
import time
import datetime

# Bibliotecas de Métricas de Machine Learning
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Parâmetros de Otimização
import warnings
%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='orange' style='font-size: 40px;'> Estrutura de um Experimento </font>
<hr style='border: 2px solid orange;'>

### 1) ATE e CATE

> 1. O primeiro passo para se tornar sólido em inferência causal é compreender profundamente o conceito de efeito causal médio (ATE — Average Treatment Effect) e efeito causal condicional (CATE — Conditional Average Treatment Effect):

- O ATE mede o impacto médio de uma intervenção sobre uma população, respondendo à pergunta: “qual o efeito médio do novo modelo de crédito sobre a taxa de default?”. 

#### $$ ATE = E[Y|T=1 - Y|T=0] $$

- Já o CATE refina essa ideia ao considerar que o efeito pode variar entre grupos de clientes com características diferentes — por exemplo, o novo modelo pode reduzir o default de forma mais acentuada entre clientes de baixa renda do que entre os de alta renda. 

#### $$ CATE_{x} = E[Y|T=1 - Y|T=0 | X = x] $$

> 2. A base teórica para esses conceitos está no Modelo de Resultados Potenciais de Rubin, que define que cada indivíduo tem dois resultados possíveis: um caso receba o tratamento (Y(1)) e outro caso não receba (Y(0)). O desafio é que nunca observamos os dois ao mesmo tempo, o que gera o chamado problema fundamental da inferência causal. Todo o arcabouço metodológico posterior — PSM, DiD, Causal Forest — é uma forma de estimar esse contrafactual ausente.

> 3. Exemplo: em um banco que implementou um novo modelo de crédito, o ATE responderia à pergunta “em média, o novo modelo reduziu o default rate da carteira?”, enquanto o CATE permitiria uma visão granular: “entre clientes de baixa renda, o novo modelo reduziu o default em 6 p.p., enquanto entre clientes de alta renda, a redução foi de apenas 2 p.p.”. Assim, o CATE revela para quem o efeito é mais forte — algo essencial em decisões de risco, marketing e política de crédito.

### 2) Experimentos Randomizados Controlados (Testes A/B)

> 1. Depois de dominar o conceito de efeito causal, o segundo passo é entender o cenário ideal: o experimento randomizado controlado, também conhecido como Teste A/B. Nele, a designação de tratamento (ex.: exposição a um novo modelo de crédito ou política de corte) é feita de forma aleatória, garantindo por construção que o grupo tratado e o grupo controle sejam comparáveis. Isso torna possível estimar o ATE de forma direta, simplesmente pela diferença entre as médias dos grupos:

#### $$ ATE = E[Y|T=1] - E[Y|T=0] $$

> 2. A análise estatística em Testes A/B envolve conceitos familiares, como intervalos de confiança, teste t, análise de poder estatístico e controle de false positives. Essa etapa é essencial porque define o padrão-ouro da inferência causal: randomizar é a forma mais limpa de estimar causalidade. Para o contexto bancário, isso é aplicável em pilotos controlados de novos modelos de crédito, ajustes de limites, políticas de cobrança ou ofertas de renegociação. Mesmo quando a randomização completa não é possível, compreender Testes A/B ajuda a avaliar o quão distante seu cenário observacional está desse ideal.

> 3. Exemplo: um banco pode aplicar um Teste A/B ao lançar um novo modelo de crédito: metade dos clientes recebe o modelo novo, metade continua com o antigo. Se o grupo tratado apresentar 3 p.p. menor taxa de default em média, isso fornece uma estimativa direta do ATE do modelo.

### 3) Propensity Score Matching e Inverse Probability Weighting

> 1. Em muitos casos, especialmente em bancos, a randomização perfeita é impossível, e o grupo Piloto acaba sendo composto por uma amostra enviesada — por exemplo, regiões com menor inadimplência ou carteiras de clientes com melhor histórico. Nesse caso, o terceiro passo é entender os métodos de ajuste observacional, sendo o Propensity Score Matching (PSM) o mais clássico. Ele consiste em estimar a probabilidade de um cliente receber o tratamento com base em suas covariáveis e, em seguida, parear clientes tratados e não tratados com propensões similares. Assim, tentamos criar um grupo controle “equivalente” ao grupo tratado, reduzindo o viés de seleção.

> 2. No entanto, o pareamento pode desperdiçar muitas observações, e por isso métodos como o Inverse Probability Weighting (IPW) — que repondera os indivíduos com base $1/P_{PSM}$ a fim de rebalancear a amostra. É muito usado para eliminar viés de seleção em Targets de Modelos

> 3. Exemplo: 

- PSM: Suponha que o banco só ofereceu um novo modelo de crédito em regiões de menor risco. O PSM permite simular um cenário comparável ao “contrafactual”, ajustando o peso dos clientes de alto risco que não receberam o novo modelo para equilibrar o grupo.

- IPW: Suponha que deseja-se criar um modelo de rendas mas só há target para o público de renda observada. O IPW permite usar o PSM e, baseado na probabilidade, criar uma amostra ponderada para eliminar viés de seleção.

### 4) Difference-in-Difference

> 1. Após dominar os métodos de pareamento, o quarto passo é estudar o Difference-in-Differences (DiD), uma técnica quase-experimental essencial quando há dados antes e depois da intervenção tanto para o grupo tratado quanto para o grupo controle. A ideia do DiD é que, se as tendências dos grupos eram paralelas antes do tratamento, qualquer diferença adicional observada após o tratamento pode ser atribuída ao efeito causal. Formalmente, o estimador é dado por:

#### $$ DiD = (Y1_{tratado} - Y0_{tratado}) - (Y1_{controle} - Y0_{controle}) $$

> 2. O DiD é extremamente poderoso porque elimina vieses fixos no tempo e permite medir mudanças relativas. É ideal para avaliar o impacto de novos modelos, políticas de aprovação, campanhas ou regras de score implementadas apenas em parte da carteira ou em regiões específicas. Um ponto crítico é sempre verificar a suposição de tendências paralelas por meio de análises gráficas ou testes placebo — isso garante a validade causal da inferência.

> 3. Exemplo: suponha que um banco introduziu um novo modelo de crédito apenas em São Paulo em janeiro, e deseja avaliar seu impacto comparando com o Rio de Janeiro, que manteve o modelo antigo. Se antes da mudança os defaults evoluíam de forma similar nas duas regiões, mas depois o default caiu mais em São Paulo, o DiD capta esse efeito diferencial.

### 5) Causal Forest

> 1. O quinto passo é compreender os métodos de Causal Machine Learning, em especial o Causal Forest e o Generalized Random Forest (GRF). Essas técnicas combinam a lógica da inferência causal com o poder preditivo do aprendizado de máquina. Enquanto as florestas tradicionais buscam minimizar o erro de previsão, o Causal Forest é projetado para identificar heterogeneidades no efeito causal, ou seja, estimar o CATE(x) — o efeito esperado do tratamento para indivíduos com características específicas. Cada árvore no Causal Forest é construída para maximizar a diferença esperada de efeitos entre subgrupos homogêneos, e a média das árvores fornece uma estimativa robusta e interpretável. No contexto bancário, isso significa descobrir “para quem” o novo modelo de crédito funciona melhor: talvez ele reduza o default rate fortemente entre autônomos, mas tenha efeito neutro em servidores públicos.

> 2. Em outras palavras, enquanto uma floresta comum responde “qual o default esperado dado a renda e a idade?”, a Causal Forest responde “qual a diferença esperada no default se esse cliente for avaliado com o novo modelo de crédito em vez do antigo?”.

> 3. Exemplo: em um banco, o Causal Forest pode mostrar que o novo modelo de crédito reduziu o default rate em 8 p.p. entre autônomos, mas teve efeito neutro em servidores públicos. Essa descoberta ajuda a direcionar políticas segmentadas — como ajustar limites, revisar regras de concessão ou oferecer produtos diferenciados — de modo orientado por evidência causal e não apenas correlação. 

# <font color='orange' style='font-size: 40px;'> Exemplo Propensity Score Matching </font>
<hr style='border: 2px solid orange;'>

> O Propensity Score Matching é uma técnica de inferência causal que busca aproximar um experimento aleatório a partir de dados observacionais. A ideia central é estimar a probabilidade de um indivíduo receber o tratamento com base em suas características observáveis (o propensity score) e, em seguida, parear indivíduos tratados e não tratados com escores semelhantes. Isso reduz o viés de seleção proveniente de diferenças observáveis entre os grupos. Contudo, o PSM só garante validade se todas as variáveis que afetam simultaneamente o tratamento e o desfecho forem observadas e incluídas no modelo — caso contrário, ainda pode haver viés de variáveis omitidas.

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

https://builtin.com/data-science/propensity-score-matching

# <font color='orange' style='font-size: 40px;'> Exemplo Diferenças-em-Diferenças </font>
<hr style='border: 2px solid orange;'>

> O método de Diferenças-em-Diferenças compara a evolução de um desfecho ao longo do tempo entre grupos tratados e não tratados. A lógica é que, na ausência do tratamento, a trajetória dos dois grupos seria paralela, e portanto qualquer divergência observada após a intervenção pode ser atribuída ao efeito causal. Essa suposição de “tendências paralelas” é crucial, pois, se não for válida, a estimação estará enviesada. O DiD é muito usado em políticas públicas e avaliação de impacto, principalmente quando há dados longitudinais ou em painel.

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

https://www.youtube.com/watch?v=j37HHDypCp8&t=51s

https://anthonypuggs.substack.com/p/python-replication-of-card-and-krueger?utm_campaign=post&utm_medium=web

https://github.com/AnthonyPuggs/CardKrueger1994Replication/tree/main

# <font color='orange' style='font-size: 40px;'> Exemplo Causal Forests </font>
<hr style='border: 2px solid orange;'>

> As Causal Forests são uma extensão das Random Forests desenvolvidas para estimar efeitos causais heterogêneos — ou seja, para identificar como o impacto de um tratamento varia entre indivíduos com diferentes características. Enquanto uma Random Forest tradicional busca prever um resultado Y, a Causal Forest procura estimar o efeito causal condicional (CATE), representado por $E[Y_{1} - Y_{0} | X=x]$, onde $Y_{1}$ e $Y_{0}$ são os resultados potenciais com e sem tratamento. O algoritmo divide recursivamente o espaço das covariáveis em regiões homogêneas em termos de efeito causal, e combina múltiplas árvores para produzir uma estimativa robusta e de baixa variância. Essa abordagem permite identificar “para quem” uma intervenção é mais efetiva, possibilitando análises segmentadas e políticas personalizadas. No contexto de risco de crédito, as Causal Forests são úteis para descobrir, por exemplo, quais grupos de clientes se beneficiam mais da adoção de um novo modelo de crédito ou política de concessão, contribuindo para decisões mais direcionadas e eficientes.

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

https://www.kaggle.com/code/carlmcbrideellis/causal-forests-double-ml-example-using-econml